In [48]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [49]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando dispositivo: {device}")

batch_size = 32
num_epochs = 64
learning_rate = 0.01
classes = ['Milho - Cercospora Gray Spot', 'Milho - Ferrugem', 'Milho - Northern Leaf Blight', 'Milho - Saudavel',
           'Soja - Masaico Amarelo', 'Soja - Sudden Death Syndrone', 'Trigo - Ferrugem Listrada', 'Trigo - Saudavel', 'Trigo - Septoria']

# Transformações para pré-processamento das imagens
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.CenterCrop(10),
    transforms.RandomResizedCrop(128, scale=(0.5, 1.0)),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Carregando os datasets
data_dir = "Data"
train_data = datasets.ImageFolder(root=f"{data_dir}/train", transform=transform)
test_data = datasets.ImageFolder(root=f"{data_dir}/test", transform=transform)

Usando dispositivo: cpu


In [50]:
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):

        super().__init__()

        # Camadas convolucionais
        self.conv1 = nn.Conv2d(3, 24, kernel_size=5, stride=1, padding=1)  # (24, 126, 126)
        self.pool = nn.MaxPool2d(2, 2)  # (24, 63, 63)
        self.conv2 = nn.Conv2d(24, 48, kernel_size=5, stride=1, padding=0)  # (48, 59, 59)
        self.pool2 = nn.MaxPool2d(2, 2)  # (48, 29.5, 29.5)
        self.conv3 = nn.Conv2d(48, 94, kernel_size=5, stride=1, padding=0)  # (94, 25,5, 25,5)
        self.pool3 = nn.MaxPool2d(2, 2)  # (94, 12,75, 12,75)

        # Camadas totalmente conectadas
        self.fc1 = nn.Linear(94 * 12 * 12, 120)  # Ajustado para 94 filtros
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, len(classes))

    def forward(self, model):
        model = F.relu(self.conv1(model))
        model = self.pool(model)
        model = F.relu(self.conv2(model))
        model = self.pool2(model)
        model = F.relu(self.conv3(model))
        model = self.pool3(model)

        model = torch.flatten(model, 1)  # Achata para a camada densa
        model = F.relu(self.fc1(model))
        model = F.relu(self.fc2(model))
        model = self.fc3(model)
        return model


In [52]:
net = NeuralNetwork().to(device)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

print(f"Modelo está na GPU? {next(net.parameters()).is_cuda}")

Modelo está na GPU? False


In [53]:
for epoch in range(num_epochs):

    # Coloca o modelo em modo de treino
    net.train()

    running_loss = 0.0

    for images, labels in train_loader:
        
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)

        optimizer.zero_grad()

        outputs = net(images) 
        loss = loss_function(outputs, labels) 

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}")

Epoch 1/64, Loss: 2.1712
Epoch 2/64, Loss: 2.0620
Epoch 3/64, Loss: 1.9786
Epoch 4/64, Loss: 1.9127
Epoch 5/64, Loss: 1.7421
Epoch 6/64, Loss: 1.6175
Epoch 7/64, Loss: 1.6458
Epoch 8/64, Loss: 1.5036
Epoch 9/64, Loss: 1.5187
Epoch 10/64, Loss: 1.3914
Epoch 11/64, Loss: 1.4455
Epoch 12/64, Loss: 1.3857
Epoch 13/64, Loss: 1.2585
Epoch 14/64, Loss: 1.2889
Epoch 15/64, Loss: 1.2591
Epoch 16/64, Loss: 1.2732
Epoch 17/64, Loss: 1.2979
Epoch 18/64, Loss: 1.2450
Epoch 19/64, Loss: 1.2061
Epoch 20/64, Loss: 1.2110
Epoch 21/64, Loss: 1.2695
Epoch 22/64, Loss: 1.2293
Epoch 23/64, Loss: 1.1754
Epoch 24/64, Loss: 1.2210
Epoch 25/64, Loss: 1.2450
Epoch 26/64, Loss: 1.1702
Epoch 27/64, Loss: 1.2918
Epoch 28/64, Loss: 1.2195
Epoch 29/64, Loss: 1.1882
Epoch 30/64, Loss: 1.2467
Epoch 31/64, Loss: 1.2046
Epoch 32/64, Loss: 1.1706
Epoch 33/64, Loss: 1.1613
Epoch 34/64, Loss: 1.1592
Epoch 35/64, Loss: 1.0900
Epoch 36/64, Loss: 1.1198
Epoch 37/64, Loss: 1.1095
Epoch 38/64, Loss: 1.1376
Epoch 39/64, Loss: 1.

# Testando o Modelo

In [54]:
# Coloca o modelo em modo de avaliação
net.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

correct = 0
total = 0

# Desativando o cálculo de gradiente para economizar memória
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = net(images)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Exibir precisão do modelo
accuracy = 100 * correct / total
print(f'Precisão do modelo no conjunto de teste: {accuracy:.2f}%')

dataiter = iter(test_loader)
images, labels = next(dataiter)

# Movendo para GPU se disponível
images, labels = images.to(device), labels.to(device)

outputs = net(images)
_, predicted = torch.max(outputs, 1)

print('Previsões:', [classes[p] for p in predicted])  # Nome das classes previstas
print('Labels reais:', [classes[l] for l in labels])  # Nome das classes corretas


Precisão do modelo no conjunto de teste: 48.04%
Previsões: ['Milho - Ferrugem', 'Milho - Saudavel', 'Milho - Northern Leaf Blight', 'Milho - Ferrugem', 'Milho - Ferrugem', 'Milho - Northern Leaf Blight', 'Milho - Cercospora Gray Spot', 'Milho - Cercospora Gray Spot', 'Milho - Ferrugem', 'Milho - Ferrugem', 'Milho - Saudavel', 'Milho - Ferrugem', 'Milho - Saudavel', 'Milho - Cercospora Gray Spot', 'Milho - Cercospora Gray Spot', 'Milho - Cercospora Gray Spot', 'Milho - Cercospora Gray Spot', 'Milho - Ferrugem', 'Milho - Ferrugem', 'Milho - Cercospora Gray Spot', 'Milho - Ferrugem', 'Milho - Cercospora Gray Spot', 'Milho - Northern Leaf Blight', 'Milho - Cercospora Gray Spot', 'Milho - Ferrugem', 'Milho - Northern Leaf Blight', 'Milho - Ferrugem', 'Milho - Ferrugem', 'Milho - Northern Leaf Blight', 'Milho - Cercospora Gray Spot', 'Milho - Cercospora Gray Spot', 'Milho - Cercospora Gray Spot']
Labels reais: ['Milho - Cercospora Gray Spot', 'Milho - Cercospora Gray Spot', 'Milho - Cercospo

## Salvando o Modelo

In [ ]:
MODEL_PATH = "modelo_cnn_01.pth"

# Salvando o modelo treinado
torch.save(net.state_dict(), MODEL_PATH)
print(f"Modelo salvo em: {MODEL_PATH}")

# Para carregar o modelo salvo posteriormente:
# 1. Criar a mesma estrutura da rede neural
model = NeuralNetwork()

# 2. Carregar os pesos salvos no modelo
model.load_state_dict(torch.load(MODEL_PATH))

# 3. Definir o modelo para modo de avaliação
model.eval()
print("Modelo carregado com sucesso!")
